## 1. Import and observe dataset
<p>We all love watching movies! There are some movies we like, some we don't. Most people have a preference for movies of a similar genre. Some of us love watching action movies, while some of us like watching horror. Some of us like watching movies that have ninjas in them, while some of us like watching superheroes.</p>
<p>Movies within a genre often share common base parameters. Consider the following two movies:</p>
<p><img style="margin:5px 20px 5px 1px; height: 250px; display: inline-block;" alt="2001: A Space Odyssey" src="https://assets.datacamp.com/production/project_648/img/movie1.jpg">
<img style="margin:5px 20px 5px 1px; height: 250px; display: inline-block;" alt="Close Encounters of the Third Kind" src="https://assets.datacamp.com/production/project_648/img/movie2.jpg"></p>
<p>Both movies, <em>2001: A Space Odyssey</em> and <em>Close Encounters of the Third Kind</em>, are movies based on aliens coming to Earth. I've seen both, and they indeed share many similarities. We could conclude that both of these fall into the same genre of movies based on intuition, but that's no fun in a data science context. In this notebook, we will quantify the similarity of movies based on their plot summaries available on IMDb and Wikipedia, then separate them into groups, also known as clusters. We'll create a dendrogram to represent how closely the movies are related to each other.</p>
<p>Let's start by importing the dataset and observing the data provided.</p>

In [200]:
# Import modules
import numpy as np
import pandas as pd
import nltk

# Set seed for reproducibility
np.random.seed(5)

# Read in IMDb and Wikipedia movie data (both in same file)
movies_df = pd.read_csv("datasets/movies.csv")

print("Number of movies loaded: %s " % (len(movies_df)))

# Display the data
movies_df

Number of movies loaded: 100 


,rank,title,genre,wiki_plot,imdb_plot
0,0,The Godfather,"[u' Crime', u' Drama']","On the day of his only daughter's wedding, Vit...","In late summer 1945, guests are gathered for t..."
1,1,The Shawshank Redemption,"[u' Crime', u' Drama']","In 1947, banker Andy Dufresne is convicted of ...","In 1947, Andy Dufresne (Tim Robbins), a banker..."
2,2,Schindler's List,"[u' Biography', u' Drama', u' History']","In 1939, the Germans move Polish Jews into the...",The relocation of Polish Jews from surrounding...
3,3,Raging Bull,"[u' Biography', u' Drama', u' Sport']","In a brief scene in 1964, an aging, overweight...","The film opens in 1964, where an older and fat..."
4,4,Casablanca,"[u' Drama', u' Romance', u' War']",It is early December 1941. American expatriate...,"In the early years of World War II, December 1..."
5,5,One Flew Over the Cuckoo's Nest,[u' Drama'],"In 1963 Oregon, Randle Patrick ""Mac"" McMurphy ...","In 1963 Oregon, Randle Patrick McMurphy (Nicho..."
6,6,Gone with the Wind,"[u' Drama', u' Romance', u' War']",\nPart 1\n \n Part 1 Part 1 \n \n On the...,"The film opens in Tara, a cotton plantation ow..."
7,7,Citizen Kane,"[u' Drama', u' Mystery']",\n\n\n\nOrson Welles as Charles Foster Kane\n\...,"It's 1941, and newspaper tycoon Charles Foster..."
8,8,The Wizard of Oz,"[u' Adventure', u' Family', u' Fantasy', u' Mu...",The film starts in sepia-tinted Kansas in the ...,Dorothy Gale (Judy Garland) is an orphaned tee...
9,9,Titanic,"[u' Drama', u' Romance']","In 1996, treasure hunter Brock Lovett and his ...","In 1996, treasure hunter Brock Lovett and his ..."


## 2. Combine Wikipedia and IMDb plot summaries
<p>The dataset we imported currently contains two columns titled <code>wiki_plot</code> and <code>imdb_plot</code>. They are the plot found for the movies on Wikipedia and IMDb, respectively. The text in the two columns is similar, however, they are often written in different tones and thus provide context on a movie in a different manner of linguistic expression. Further, sometimes the text in one column may mention a feature of the plot that is not present in the other column. For example, consider the following plot extracts from <em>The Godfather</em>:</p>
<ul>
<li>Wikipedia: "On the day of his only daughter's wedding, Vito Corleone"</li>
<li>IMDb: "In late summer 1945, guests are gathered for the wedding reception of Don Vito Corleone's daughter Connie"</li>
</ul>
<p>While the Wikipedia plot only mentions it is the day of the daughter's wedding, the IMDb plot also mentions the year of the scene and the name of the daughter. </p>
<p>Let's combine both the columns to avoid the overheads in computation associated with extra columns to process.</p>

In [202]:
# Combine wiki_plot and imdb_plot into a single column
movies_df["plot"] = movies_df["wiki_plot"].astype(str) + "\n" + \
                 movies_df["imdb_plot"].astype(str)

# Inspect the new DataFrame
movies_df.head()

,rank,title,genre,wiki_plot,imdb_plot,plot
0,0,The Godfather,"[u' Crime', u' Drama']","On the day of his only daughter's wedding, Vit...","In late summer 1945, guests are gathered for t...","On the day of his only daughter's wedding, Vit..."
1,1,The Shawshank Redemption,"[u' Crime', u' Drama']","In 1947, banker Andy Dufresne is convicted of ...","In 1947, Andy Dufresne (Tim Robbins), a banker...","In 1947, banker Andy Dufresne is convicted of ..."
2,2,Schindler's List,"[u' Biography', u' Drama', u' History']","In 1939, the Germans move Polish Jews into the...",The relocation of Polish Jews from surrounding...,"In 1939, the Germans move Polish Jews into the..."
3,3,Raging Bull,"[u' Biography', u' Drama', u' Sport']","In a brief scene in 1964, an aging, overweight...","The film opens in 1964, where an older and fat...","In a brief scene in 1964, an aging, overweight..."
4,4,Casablanca,"[u' Drama', u' Romance', u' War']",It is early December 1941. American expatriate...,"In the early years of World War II, December 1...",It is early December 1941. American expatriate...


## 3. Tokenization
<p>Tokenization is the process  by which we break down articles into individual sentences or words, as needed. Besides the tokenization method provided by NLTK, we might have to perform additional filtration to remove tokens which are entirely numeric values or punctuation.</p>
<p>While a program may fail to build context from "While waiting at a bus stop in 1981" (<em>Forrest Gump</em>), because this string would not match in any dictionary, it is possible to build context from the words "while", "waiting" or "bus" because they are present in the English dictionary. </p>
<p>Let us perform tokenization on a small extract from <em>The Godfather</em>.</p>

In [204]:
# Tokenize a paragraph into sentences and store in sent_tokenized
sent_tokenized = [sent for sent in nltk.sent_tokenize("""
                        Today (May 19, 2016) is his only daughter's wedding. 
                        Vito Corleone is the Godfather.
                        """)]

# Word Tokenize first sentence from sent_tokenized, save as words_tokenized
words_tokenized = [word for word in nltk.word_tokenize(sent_tokenized[0])]

# Remove tokens that do not contain any letters from words_tokenized
import re

filtered = [word for word in words_tokenized if re.search("[a-zA-Z]", word)]

# Display filtered words to observe words after tokenization
filtered

['Today', 'May', 'is', 'his', 'only', 'daughter', "'s", 'wedding']

## 4. Stemming
<p>Stemming is the process by which we bring down a word from its different forms to the root word. This helps us establish meaning to different forms of the same words without having to deal with each form separately. For example, the words 'fishing', 'fished', and 'fisher' all get stemmed to the word 'fish'.</p>
<p>Consider the following sentences:</p>
<ul>
<li>"Young William Wallace witnesses the treachery of Longshanks" ~ <em>Gladiator</em></li>
<li>"escapes to the city walls only to witness Cicero's death" ~ <em>Braveheart</em></li>
</ul>
<p>Instead of building separate dictionary entries for both witnesses and witness, which mean the same thing outside of quantity, stemming them reduces them to 'wit'.</p>
<p>There are different algorithms available for stemming such as the Porter Stemmer, Snowball Stemmer, etc. We shall use the Snowball Stemmer.</p>

In [206]:
# Import the SnowballStemmer to perform stemming
# ... YOUR CODE FOR TASK 4 ...
from nltk.stem.snowball import SnowballStemmer
# Create an English language SnowballStemmer object
stemmer = SnowballStemmer("english")

# Print filtered to observe words without stemming
print("Without stemming: ", filtered)

# Stem the words from filtered and store in stemmed_words
stemmed_words = [ stemmer.stem(word) for word in filtered]

# Print the stemmed_words to observe words after stemming
print("After stemming:   ", stemmed_words)

Without stemming:  ['Today', 'May', 'is', 'his', 'only', 'daughter', "'s", 'wedding']
After stemming:    ['today', 'may', 'is', 'his', 'onli', 'daughter', "'s", 'wed']


## 5. Club together Tokenize & Stem
<p>We are now able to tokenize and stem sentences. But we may have to use the two functions repeatedly one after the other to handle a large amount of data, hence we can think of wrapping them in a function and passing the text to be tokenized and stemmed as the function argument. Then we can pass the new wrapping function, which shall perform both tokenizing and stemming instead of just tokenizing, as the tokenizer argument while creating the TF-IDF vector of the text.  </p>
<p>What difference does it make though? Consider the sentence from the plot of <em>The Godfather</em>: "Today (May 19, 2016) is his only daughter's wedding." If we do a 'tokenize-only' for this sentence, we have the following result:</p>
<blockquote>
  <p>'today', 'may', 'is', 'his', 'only', 'daughter', "'s", 'wedding'</p>
</blockquote>
<p>But when we do a 'tokenize-and-stem' operation we get:</p>
<blockquote>
  <p>'today', 'may', 'is', 'his', 'onli', 'daughter', "'s", 'wed'</p>
</blockquote>
<p>All the words are in their root form, which will lead to a better establishment of meaning as some of the non-root forms may not be present in the NLTK training corpus.</p>

In [208]:
# Define a function to perform both stemming and tokenization
def tokenize_and_stem(text):
    
    # Tokenize by sentence, then by word

    tokens = [y for x in nltk.sent_tokenize(text) for y in nltk.word_tokenize(x)]
    #[y for text in nltk.sent_tokenize(text) for y in nltk.word_tokenize(text)]
    print(tokens)
    
    # Filter out raw tokens to remove noise
    filtered_tokens = [token for token in tokens if re.search('[a-zA-Z]', token)]
    #filtered_tokens = [token for token in tokens if re.search('[a-zA-Z]', token)]
    
    # Stem the filtered_tokens
    stems = stemmer.stem(text)
    
    return stems

words_stemmed = tokenize_and_stem("Today (May 19, 2016) is his only daughter's wedding.")
print(words_stemmed)

['Today', '(', 'May', '19', ',', '2016', ')', 'is', 'his', 'only', 'daughter', "'s", 'wedding', '.']
today (may 19, 2016) is his only daughter's wedding.


## 6. Create TfidfVectorizer
<p>Computers do not <em>understand</em> text. These are machines only capable of understanding numbers and performing numerical computation. Hence, we must convert our textual plot summaries to numbers for the computer to be able to extract meaning from them. One simple method of doing this would be to count all the occurrences of each word in the entire vocabulary and return the counts in a vector. Enter <code>CountVectorizer</code>.</p>
<p>Consider the word 'the'. It appears quite frequently in almost all movie plots and will have a high count in each case. But obviously, it isn't the theme of all the movies! <a href="https://campus.datacamp.com/courses/natural-language-processing-fundamentals-in-python/simple-topic-identification?ex=11">Term Frequency-Inverse Document Frequency</a> (TF-IDF) is one method which overcomes the shortcomings of <code>CountVectorizer</code>. The Term Frequency of a word is the measure of how often it appears in a document, while the Inverse Document Frequency is the parameter which reduces the importance of a word if it frequently appears in several documents.</p>
<p>For example, when we apply the TF-IDF on the first 3 sentences from the plot of <em>The Wizard of Oz</em>, we are told that the most important word there is 'Toto', the pet dog of the lead character. This is because the movie begins with 'Toto' biting someone due to which the journey of Oz begins!</p>
<p>In simplest terms, TF-IDF recognizes words which are unique and important to any given document. Let's create one for our purposes.</p>

In [210]:
# Import TfidfVectorizer to create TF-IDF vectors
# ... YOUR CODE FOR TASK 6 ...
from sklearn.feature_extraction.text import TfidfVectorizer 

# Instantiate TfidfVectorizer object with stopwords and tokenizer
# parameters for efficient processing of text
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem,
                                 ngram_range=(1,3))

## 7. Fit transform TfidfVectorizer
<p>Once we create a TF-IDF Vectorizer, we must fit the text to it and then transform the text to produce the corresponding numeric form of the data which the computer will be able to understand and derive meaning from. To do this, we use the <code>fit_transform()</code> method of the <code>TfidfVectorizer</code> object. </p>
<p>If we observe the <code>TfidfVectorizer</code> object we created, we come across a parameter stopwords. 'stopwords' are those words in a given text which do not contribute considerably towards the meaning of the sentence and are generally grammatical filler words. For example, in the sentence 'Dorothy Gale lives with her dog Toto on the farm of her Aunt Em and Uncle Henry', we could drop the words 'her' and 'the', and still have a similar overall meaning to the sentence. Thus, 'her' and 'the' are stopwords and can be conveniently dropped from the sentence. </p>
<p>On setting the stopwords to 'english', we direct the vectorizer to drop all stopwords from a pre-defined list of English language stopwords present in the nltk module. Another parameter, <code>ngram_range</code>, defines the length of the ngrams to be formed while vectorizing the text.</p>

In [212]:
# Fit and transform the tfidf_vectorizer with the "plot" of each movie
# to create a vector representation of the plot summaries
tfidf_matrix = tfidf_vectorizer.fit_transform([x for x in movies_df["plot"]])

print(tfidf_matrix.shape)

['four']
['ie']
['further']
['next']
['have']
['before']
['yourself']
['too']
['is']
['it']
['under']
['hasnt']
['sometime']
['another']
['somehow']
['anyhow']
['co']
['her']
['nowhere']
['eleven']
['about']
['hereby']
['us']
['formerly']
['both']
['herein']
['rather']
['nor']
['or']
['serious']
['empty']
['she']
['which']
['me']
['sixty']
['find']
['themselves']
['am']
['latterly']
['his']
['you']
['whatever']
['these']
['wherever']
['with']
['onto']
['two']
['whereas']
['some']
['behind']
['upon']
['the']
['many']
['being']
['former']
['hence']
['besides']
['seeming']
['together']
['whole']
['such']
['therein']
['amount']
['can', 'not']
['full']
['show']
['inc']
['fifteen']
['indeed']
['moreover']
['were']
['anyway']
['should']
['whither']
['yet']
['beyond']
['namely']
['yours']
['fire']
['again']
['per']
['nobody']
['amongst']
['becoming']
['here']
['has']
['among']
['latter']
['move']
['fifty']
['mine']
['re']
['through']
['will']
['also']
['thick']
['afterwards']
['herself']
['out

['it', 'is', 'early', 'december', '1941.', 'american', 'expatriate', 'rick', 'blaine', 'is', 'the', 'proprietor', 'of', 'an', 'upscale', 'nightclub', 'and', 'gambling', 'den', 'in', 'casablanca', '.', '``', 'rick', "'s", 'café', 'américain', "''", 'attracts', 'a', 'varied', 'clientele', ':', 'vichy', 'french', ',', 'italian', ',', 'and', 'german', 'officials', ';', 'refugees', 'desperate', 'to', 'reach', 'the', 'still', 'neutral', 'united', 'states', ';', 'and', 'those', 'who', 'prey', 'on', 'them', '.', 'although', 'rick', 'professes', 'to', 'be', 'neutral', 'in', 'all', 'matters', ',', 'it', 'is', 'later', 'revealed', 'he', 'ran', 'guns', 'to', 'ethiopia', 'during', 'its', 'war', 'with', 'italy', 'and', 'fought', 'on', 'the', 'loyalist', 'side', 'against', 'the', 'fascist', 'nationalists', 'in', 'the', 'spanish', 'civil', 'war', '.', 'it', 'is', 'early', 'december', '1941.', 'american', 'expatriate', 'expatriate', 'rick', 'blaine', 'is', 'the', 'proprietor', 'of', 'an', 'upscale', 'n

['the', 'film', 'starts', 'in', 'sepia-tinted', 'kansas', 'in', 'the', 'early', '1900s', '.', 'dorothy', 'gale', 'lives', 'with', 'her', 'dog', 'toto', 'on', 'the', 'farm', 'of', 'her', 'aunt', 'em', 'and', 'uncle', 'henry', '.', 'dorothy', 'and', 'toto', 'get', 'in', 'trouble', 'with', 'a', 'cruel', 'neighbor', ',', 'miss', 'almira', 'gulch', ',', 'when', 'toto', 'bites', 'her', '.', 'however', ',', 'dorothy', "'s", 'family', 'and', 'the', 'farmhands', 'are', 'all', 'too', 'busy', 'to', 'pay', 'attention', 'to', 'her', '.', 'miss', 'gulch', 'arrives', 'with', 'permission', 'from', 'the', 'sheriff', 'to', 'have', 'toto', 'be', 'euthanized', '.', 'he', 'is', 'taken', 'away', ',', 'but', 'escapes', 'and', 'returns', 'to', 'dorothy', ';', 'she', 'then', 'decides', 'to', 'run', 'away', 'from', 'home', 'with', 'toto', 'to', 'escape', 'miss', 'gulch', '.', 'they', 'meet', 'professor', 'marvel', ',', 'a', 'phony', 'fortune', 'teller', ',', 'who', 'realizes', 'dorothy', 'has', 'run', 'away', '

['at', 'a', 'sunset', 'boulevard', 'mansion', ',', 'the', 'body', 'of', 'joe', 'gillis', 'floats', 'in', 'the', 'swimming', 'pool', '.', 'in', 'a', 'flashback', ',', 'joe', 'relates', 'the', 'events', 'leading', 'to', 'his', 'death', '.', 'at', 'a', 'sunset', 'boulevard', 'sunset', 'boulevard', 'mansion', ',', 'the', 'body', 'of', 'joe', 'gillis', 'floats', 'in', 'the', 'swimming', 'pool', '.', 'in', 'a', 'flashback', 'flashback', ',', 'joe', 'relates', 'the', 'events', 'leading', 'to', 'his', 'death', '.', 'six', 'months', 'earlier', ',', 'down-on-his-luck', 'screenwriter', 'joe', 'tries', 'selling', 'paramount', 'pictures', 'producer', 'sheldrake', 'on', 'a', 'story', 'he', 'submitted', '.', 'script', 'reader', 'betty', 'schaefer', 'harshly', 'critiques', 'it', ',', 'unaware', 'that', 'joe', 'is', 'listening', '.', 'later', ',', 'while', 'fleeing', 'from', 'repossession', 'men', 'seeking', 'his', 'car', ',', 'joe', 'turns', 'into', 'the', 'driveway', 'of', 'a', 'seemingly', 'deserted

[']', ']', 'in', 'the', 'west', 'side', "'s", 'lincoln', 'square', 'neighborhood', 'in', 'manhattan', ',', 'there', 'is', 'tension', 'between', 'a', 'caucasian', 'gang', ',', 'the', 'jets', ',', 'led', 'by', 'riff', ',', 'and', 'a', 'puerto', 'rican', 'gang', 'of', 'immigrants', ',', 'the', 'sharks', ',', 'led', 'by', 'bernardo', '.', 'after', 'a', 'brawl', 'erupts', '(', '``', 'prologue', "''", ')', ',', 'lieutenant', 'schrank', 'and', 'officer', 'krupke', 'arrive', 'and', 'break', 'it', 'up', '.', 'the', 'jets', 'decide', 'to', 'challenge', 'the', 'sharks', 'to', 'a', 'rumble', 'at', 'an', 'upcoming', 'dance', 'for', 'neighborhood', 'control', '.', 'in', 'the', 'west', 'side', 'west', 'side', "'s", 'lincoln', 'square', 'lincoln', 'square', 'neighborhood', 'in', 'manhattan', 'manhattan', ',', 'there', 'is', 'tension', 'between', 'a', 'caucasian', 'gang', ',', 'the', 'jets', ',', 'led', 'by', 'riff', ',', 'and', 'a', 'puerto', 'rican', 'puerto', 'rican', 'gang', 'of', 'immigrants', ','

...

['in', 'the', 'sonoran', 'desert', ',', 'french', 'scientist', 'claude', 'lacombe', 'and', 'his', 'american', 'interpreter', ',', 'mapmaker', 'david', 'laughlin', ',', 'along', 'with', 'other', 'government', 'scientific', 'researchers', ',', 'discover', 'flight', '19', ',', 'a', 'squadron', 'of', 'grumman', 'tbm', 'avengers', 'that', 'went', 'missing', 'more', 'than', '30', 'years', 'earlier', '.', 'the', 'planes', 'are', 'intact', 'and', 'operational', ',', 'but', 'there', 'is', 'no', 'sign', 'of', 'the', 'pilots', '.', 'an', 'old', 'man', 'who', 'witnessed', 'the', 'event', 'claimed', '``', 'the', 'sun', 'came', 'out', 'at', 'night', ',', 'and', 'sang', 'to', 'him', '.', "''", 'they', 'also', 'find', 'a', 'lost', 'cargo', 'ship', 'in', 'the', 'gobi', 'desert', 'named', 'ss', 'cotopaxi', '.', 'at', 'an', 'air', 'traffic', 'control', 'center', 'in', 'indianapolis', ',', 'a', 'controller', 'listens', 'as', 'two', 'airline', 'flights', 'narrowly', 'avoid', 'a', 'mid-air', 'collision', 'w

['in', 'late', 'august', '1962', 'recent', 'high', 'school', 'graduates', 'and', 'longtime', 'friends', ',', 'curt', 'henderson', 'and', 'steve', 'bolander', ',', 'meet', 'john', 'milner', 'and', 'terry', '``', 'the', 'toad', "''", 'fields', 'at', 'the', 'local', 'mel', "'s", 'drive-in', 'parking', 'lot', '.', 'despite', 'receiving', 'a', '$', '2,000', 'scholarship', 'from', 'the', 'local', 'moose', 'lodge', ',', 'curt', 'is', 'undecided', 'if', 'he', 'wants', 'to', 'leave', 'the', 'next', 'morning', 'with', 'steve', 'to', 'go', 'to', 'the', 'northeastern', 'united', 'states', 'to', 'begin', 'college', '.', 'steve', 'lets', 'toad', 'borrow', 'his', '1958', 'chevy', 'impala', 'for', 'the', 'evening', ',', 'and', 'while', 'he', 'will', 'be', 'away', 'at', 'college', '.', 'steve', "'s", 'girlfriend', ',', 'laurie', ',', 'who', 'also', 'is', 'curt', "'s", 'younger', 'sister', ',', 'is', 'unsure', 'of', 'steve', "'s", 'leaving', ',', 'to', 'which', 'he', 'suggests', 'they', 'see', 'other', 

['in', '1539', 'the', 'knight', 'templars', 'of', 'malta', ',', 'paid', 'tribute', 'to', 'charles', 'v', 'of', 'spain', ',', 'by', 'sending', 'him', 'a', 'golden', 'falcon', 'encrusted', 'from', 'beak', 'to', 'claw', 'with', 'rarest', 'jewels—but', 'pirates', 'seized', 'the', 'galley', 'carrying', 'this', 'priceless', 'token', 'and', 'the', 'fate', 'of', 'the', 'maltese', 'falcon', 'remains', 'a', 'mystery', 'to', 'this', 'day', '—', '–', 'introductory', 'text', 'appearing', 'after', 'the', 'film', "'s", 'opening', 'credits', '[', '7', ']', 'in', '1539', 'the', 'knight', 'templars', 'of', 'malta', ',', 'paid', 'tribute', 'to', 'charles', 'v', 'of', 'spain', ',', 'by', 'sending', 'him', 'a', 'golden', 'falcon', 'encrusted', 'from', 'beak', 'to', 'claw', 'with', 'rarest', 'jewels—but', 'pirates', 'seized', 'the', 'galley', 'carrying', 'this', 'priceless', 'token', 'and', 'the', 'fate', 'of', 'the', 'maltese', 'falcon', 'remains', 'a', 'mystery', 'to', 'this', 'day', '—', 'in', '1539', 't

['james', 'stewart', 'as', 'l.b', '.', 'jefferies', 'james', 'stewart', 'as', 'l.b', '.', 'jefferies', 'james', 'stewart', 'as', 'l.b', '.', 'jefferies', 'james', 'stewart', 'james', 'stewart', 'as', 'l.b', '.', 'jefferies', 'after', 'breaking', 'his', 'leg', 'photographing', 'a', 'racetrack', 'accident', ',', 'professional', 'photographer', 'l.b', '.', '``', 'jeff', "''", 'jefferies', '(', 'james', 'stewart', ')', 'is', 'confined', 'to', 'his', 'greenwich', 'village', 'apartment', ',', 'using', 'a', 'wheelchair', 'while', 'he', 'recuperates', '.', 'his', 'rear', 'window', 'looks', 'out', 'onto', 'a', 'small', 'courtyard', 'and', 'several', 'other', 'apartments', '.', 'during', 'a', 'summer', 'heat', 'wave', ',', 'he', 'passes', 'the', 'time', 'by', 'watching', 'his', 'neighbors', ',', 'who', 'keep', 'their', 'windows', 'open', 'to', 'stay', 'cool', '.', 'the', 'tenants', 'he', 'can', 'see', 'include', 'a', 'dancer', 'he', 'nicknames', '``', 'miss', 'torso', "''", ',', 'a', 'lonely', '

['in', 'the', 'early', 'days', 'of', 'world', 'war', 'ii', ',', 'cohan', 'comes', 'out', 'of', 'retirement', 'to', 'star', 'as', 'president', 'roosevelt', 'in', 'the', 'rodgers', 'and', 'hart', 'musical', 'i', "'d", 'rather', 'be', 'right', '.', 'on', 'the', 'first', 'night', ',', 'he', 'is', 'summoned', 'to', 'meet', 'the', 'president', 'at', 'the', 'white', 'house', ',', 'who', 'presents', 'him', 'with', 'a', 'congressional', 'gold', 'medal', '(', 'in', 'fact', ',', 'this', 'happened', 'several', 'years', 'previously', ')', '.', 'cohan', 'is', 'overcome', 'and', 'chats', 'with', 'roosevelt', ',', 'recalling', 'his', 'early', 'days', 'on', 'the', 'stage', '.', 'the', 'film', 'flashes', 'back', 'to', 'his', 'supposed', 'birth', 'on', 'july', '4', ',', 'whilst', 'his', 'father', 'is', 'performing', 'on', 'the', 'vaudeville', 'stage', '.', 'in', 'the', 'early', 'days', 'of', 'world', 'war', 'ii', 'world', 'war', 'ii', ',', 'cohan', 'comes', 'out', 'of', 'retirement', 'to', 'star', 'as', 

## 8. Import KMeans and create clusters
<p>To determine how closely one movie is related to the other by the help of unsupervised learning, we can use clustering techniques. Clustering is the method of grouping together a number of items such that they exhibit similar properties. According to the measure of similarity desired, a given sample of items can have one or more clusters. </p>
<p>A good basis of clustering in our dataset could be the genre of the movies. Say we could have a cluster '0' which holds movies of the 'Drama' genre. We would expect movies like <em>Chinatown</em> or <em>Psycho</em> to belong to this cluster. Similarly, the cluster '1' in this project holds movies which belong to the 'Adventure' genre (<em>Lawrence of Arabia</em> and the <em>Raiders of the Lost Ark</em>, for example).</p>
<p>K-means is an algorithm which helps us to implement clustering in Python. The name derives from its method of implementation: the given sample is divided into <b><i>K</i></b> clusters where each cluster is denoted by the <b><i>mean</i></b> of all the items lying in that cluster. </p>
<p>We get the following distribution for the clusters:</p>
<p><img src="https://assets.datacamp.com/production/project_648/img/bar_clusters.png" alt="bar graph of clusters"></p>

In [214]:
# Import k-means to perform clusters
from sklearn.cluster import KMeans

# Create a KMeans object with 5 clusters and save as km
km = KMeans(n_clusters=5)

# Fit the k-means object with tfidf_matrix
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

# Create a column cluster to denote the generated cluster for each movie
movies_df["cluster"] = clusters

# Display number of films per cluster (clusters from 0 to 4)
movies_df['cluster'].value_counts() 

2    64
0    18
4     7
1     6
3     5
Name: cluster, dtype: int64

## 9. Calculate similarity distance
<p>Consider the following two sentences from the movie <em>The Wizard of Oz</em>: </p>
<blockquote>
  <p>"they find in the Emerald City"</p>
  <p>"they finally reach the Emerald City"</p>
</blockquote>
<p>If we put the above sentences in a <code>CountVectorizer</code>, the vocabulary produced would be "they, find, in, the, Emerald, City, finally, reach" and the vectors for each sentence would be as follows: </p>
<blockquote>
  <p>1, 1, 1, 1, 1, 1, 0, 0</p>
  <p>1, 0, 0, 1, 1, 1, 1, 1</p>
</blockquote>
<p>When we calculate the cosine angle formed between the vectors represented by the above, we get a score of 0.667. This means the above sentences are very closely related. <em>Similarity distance</em> is 1 - <a href="https://en.wikipedia.org/wiki/Cosine_similarity">cosine similarity angle</a>. This follows from that if the vectors are similar, the cosine of their angle would be 1 and hence, the distance between then would be 1 - 1 = 0.</p>
<p>Let's calculate the similarity distance for all of our movies.</p>

In [216]:
# Import cosine_similarity to calculate similarity of movie plots
from sklearn.metrics.pairwise import cosine_similarity

# Calculate the similarity distance
similarity_distance = 1 - cosine_similarity(tfidf_matrix)

## 10. Import Matplotlib, Linkage, and Dendrograms
<p>We shall now create a tree-like diagram (called a dendrogram) of the movie titles to help us understand the level of similarity between them visually. Dendrograms help visualize the results of hierarchical clustering, which is an alternative to k-means clustering. Two pairs of movies at the same level of hierarchical clustering are expected to have similar strength of similarity between the corresponding pairs of movies. For example, the movie <em>Fargo</em> would be as similar to <em>North By Northwest</em> as the movie <em>Platoon</em> is to <em>Saving Private Ryan</em>, given both the pairs exhibit the same level of the hierarchy.</p>
<p>Let's import the modules we'll need to create our dendrogram.</p>

In [218]:
# Import matplotlib.pyplot for plotting graphs
# ... YOUR CODE FOR TASK 10 ...
import matplotlib.pyplot as plt
# Configure matplotlib to display the output inline
%matplotlib inline

# Import modules necessary to plot dendrogram
# ... YOUR CODE FOR TASK 10 ...
from scipy.cluster.hierarchy import linkage, dendrogram

## 11. Create merging and plot dendrogram
<p>We shall plot a dendrogram of the movies whose similarity measure will be given by the similarity distance we previously calculated. The lower the similarity distance between any two movies, the lower their linkage will make an intercept on the y-axis. For instance, the lowest dendrogram linkage we shall discover will be between the movies, <em>It's a Wonderful Life</em> and <em>A Place in the Sun</em>. This indicates that the movies are very similar to each other in their plots.</p>

In [220]:
# Create mergings matrix 
mergings = linkage(similarity_distance, method='complete')

# Plot the dendrogram, using title as label column
dendrogram_ = dendrogram(mergings,
               labels=[x for x in movies_df["title"]],
               leaf_rotation=90,
               leaf_font_size=16,
)

# Adjust the plot
fig = plt.gcf()
_ = [lbl.set_color('r') for lbl in plt.gca().get_xmajorticklabels()]
fig.set_size_inches(108, 21)

# Show the plotted dendrogram
plt.show()

KeyboardInterrupt: 

## 12. Which movies are most similar?
<p>We can now determine the similarity between movies based on their plots! To wrap up, let's answer one final question: which movie is most similar to the movie <em>Braveheart</em>?</p>

In [ ]:
# Answer the question 

ans = "Jaws"
print(ans)